In [17]:
import ROOT
import numpy as np
import math
import copy
import os
%run ../../utils/helper.ipynb

In [2]:
# Define momentum ranges
momentum_ranges = np.arange(0,10,1)
num_ranges = len(momentum_ranges)

In [3]:
# Define plot marker styles and colors
marker_styles = [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
marker_colors = [1]

In [24]:
def plot_contamination(project_dir, channel, version, cut_name,var,vmin,vmax):
    if(not os.path.exists(f"{project_dir}/systematics/tfr_contamination")):
        os.makedirs(f"{project_dir}/systematics/tfr_contamination")
    if(not os.path.exists(f"{project_dir}/systematics/tfr_contamination/{channel}")):
        os.makedirs(f"{project_dir}/systematics/tfr_contamination/{channel}")
    
    # Open input file and get TTree
    if(version=="MC_RGA_inbending"):
        suffix="MC_RGA_30*.root"
    elif(version=="MC_RGA_outbending"):
        suffix="MC_RGA_3313_*.root"
    
    tree = ROOT.TChain("dihadron")
    tree.Add(f"{project_dir}/volatile/data/{channel}/{suffix}")
    CHANNEL=channel.replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
    # Get cut from channel
    cut_vars,cut_mins,cut_maxs,_=get_cuts(channel,cut_name)
    cuts=[]
    for cv,cl,ch in zip(cut_vars,cut_mins,cut_maxs):
        cuts.append(f"{cv}>{cl}&&{cv}<{ch}")
    cut="&&".join(cuts)
    
    # Define lists to store TH1F objects
    hist_list = []
    # Create TCanvas and TLegend
    canvas = ROOT.TCanvas("canvas", "TFR Contamination vs. Theta",800,600)
    legend = ROOT.TLegend(0.6, 0.6, 0.9, 0.9)
    ROOT.gStyle.SetOptStat(0)

    # Create TH1F for this momentum range
    hist_false = ROOT.TH1F("hist_false", "TFR Contamination vs. {} ( {} dihadrons)".format(var,CHANNEL), 15, vmin, vmax)
    hist_true = ROOT.TH1F("hist_true", "TFR Contamination vs. {} ( {} dihadrons)".format(var,CHANNEL), 15, vmin, vmax)
    # Fill histograms with contamination percentages
    tree.Draw("{} >> hist_false".format(var), "{}&&(is_CFR_1==0||is_CFR_2==0)".format(cut), "goff")
    tree.Draw("{} >> hist_true".format(var), "{}".format(cut), "goff")
    # Divide histograms to get contamination percentage
    #hist_false.Divide(hist_true)
    # Propagate uncertainty
    for i in range(hist_false.GetNbinsX()):
        a = hist_false.GetBinContent(i)
        b = hist_true.GetBinContent(i)
        sigma_a = hist_false.GetBinError(i)
        sigma_b = hist_true.GetBinError(i)
        if b != 0 and a!=0:
            c = a/b
            sigma_c = c * math.sqrt((sigma_a/a)**2 + (sigma_b/b)**2)
        else:
            c = 0
            sigma_c = 0
        hist_false.SetBinContent(i, c)
        hist_false.SetBinError(i, sigma_c)
    # Add TH1F to list
    hist_list.append(hist_false)
    # Set histogram style and add to legend
    hist_list[-1].SetMarkerColor(1)
    hist_list[-1].SetMarkerStyle(20)
    hist_list[-1].SetMarkerSize(1.5)
    # Draw TH1F on canvas
    hist_list[-1].Draw("E1 P")
    
    # Set canvas properties and draw legend
    hist_list[0].GetYaxis().SetRangeUser(0.00001,1)
    canvas.SetLogy(1)
    canvas.SetGrid()
    # Set axis labels and draw on canvas
    hist_list[0].GetXaxis().SetTitle(var)
    hist_list[0].GetYaxis().SetTitle("Contamination (%)")
    canvas.SaveAs(f"{project_dir}/systematics/tfr_contamination/{channel}/figure_{var}_{version}.png")

In [25]:
project_dir="/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v1"
versions = ["MC_RGA_inbending","MC_RGA_outbending"]
channels = ["piplus_piminus", "piplus_pi0", "piplus_piplus","pi0_pi0","piminus_pi0", "piminus_piminus"]
cut_names = ["v0","v0","v0","v0","v0","v0"]

In [29]:
for version in versions:
    for channel,cut_name in zip(channels,cut_names):
        print(version,channel)
        plot_contamination(project_dir,channel,version,cut_name,"phi_h",-3.1415,3.1415)

MC_RGA_inbending piplus_piminus
MC_RGA_inbending piplus_pi0
MC_RGA_inbending piplus_piplus
MC_RGA_inbending pi0_pi0
MC_RGA_inbending piminus_pi0
MC_RGA_inbending piminus_piminus
MC_RGA_outbending piplus_piminus
MC_RGA_outbending piplus_pi0
MC_RGA_outbending piplus_piplus
MC_RGA_outbending pi0_pi0
MC_RGA_outbending piminus_pi0
MC_RGA_outbending piminus_piminus


Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v1/systematics/tfr_contamination/piplus_piminus/figure_phi_h_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v1/systematics/tfr_contamination/piplus_pi0/figure_phi_h_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v1/systematics/tfr_contamination/piplus_piplus/figure_phi_h_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v1/systematics/tfr_contamination/pi0_pi0/figure_phi_h_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v1/systematics/tfr_contamination/piminus_pi0/figure_phi_h_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work

In [16]:
import os

# Define the output filename for the LaTeX file
latex_filename = f"{project_dir}/systematics/false_id/plots.tex"

# Open the LaTeX file for writing
with open(latex_filename, "w") as f:
    # Write the LaTeX document preamble
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage{graphicx}\n")
    #f.write("\\usepackage{caption}\n")
    f.write("\\usepackage{geometry}\n")
    f.write("\\usepackage{fancyhdr}\n")

    f.write("\\geometry{margin=2cm}\n") # Set the margin to 1 cm
    f.write("\\usepackage[font=large]{caption}\n") # Set the caption font size to large
    f.write("\\begin{document}\n")
    f.write("\\pagestyle{fancy}\n")
    f.write("\\fancyhf{}\n")
    f.write("\\fancyhead[L]{\\leftmark\\,\\,\\,\\,\\rightmark}\n")
    f.write("\\fancyhead[R]{\\thepage}\n")
    # Add a table of contents
    f.write("\\tableofcontents\n")
    # Loop over all channels
    for channel in channels:
        CHANNEL=channel.replace("piplus","$\\pi^{+}$").replace("piminus","$\\pi^{-}$").replace("pi0","$\\pi^{0}$").replace("_","")
        f.write(f"\\section{{{CHANNEL} dihadrons}}") # Add a section for the channel
        
        # Loop over all versions
        for version in versions:
            VERSION=version.replace("_"," ")
            f.write(f"\\subsection{{{VERSION}}}") # Add a subsection for the version
            reco_info,true_info=get_ri_ti(channel)
            # Loop over all binstructs
            for ri,ti in zip(reco_info,true_info):
                # Define the plot filename and caption
                plotname = f'{project_dir}/systematics/false_id/{channel}/figure_{ri[0]}_as_{ti[0]}_{version}.png'
                caption = f"NEWLINE{CHANNEL}NEWLINEMonte Carlo Version={version}"
                #plotname = plotname.replace("_","\_")
                caption = caption.replace("_","\_").replace("NEWLINE","\\\\")
                # Write the LaTeX code for the plot and caption
                f.write("\\begin{figure}[ht]\n")
                f.write(f"\\includegraphics[width=\\textwidth]{{{plotname}}}\n")
                f.write(f"\\captionsetup{{font=large}}\n") # Set the caption font size to large
                f.write(f"\\caption{{{caption}}}\n")
                f.write("\\end{figure}\n")
            f.write("\\clearpage")
    # Write the LaTeX document postamble
    f.write("\\end{document}\n")
    
# Get the directory part of the LaTeX filename
latex_dirname = os.path.dirname(latex_filename)

# Save the current working directory
cwd = os.getcwd()

# Change the current working directory to the LaTeX directory
os.chdir(latex_dirname)

# Compile the LaTeX file into a PDF
os.system(f"pdflatex {latex_filename}")
os.system(f"pdflatex {latex_filename}")

# Change the current working directory back to the original
os.chdir(cwd)

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode

(/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/fa
lse_id/plots.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/trig.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-cfg/graphics.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-def/pdftex.def)))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/generic/ifte